# Settings

In [1]:
IN_FILENAME = "../data/processed/calculated.xlsx"
OUT_FILENAME = "../data/processed/filtered.xlsx"

SEPARATOR = "\n"

### adjustments

In [2]:
import re

fmt = re.search("[.][a-z]+$", OUT_FILENAME)
if fmt == None: raise Exception("have to choose a format (xlsx | csv)")
fmt = fmt.group()[1:]

# Prevs

### Imports

In [3]:
import pandas as pd
import numpy as np
from collections import namedtuple
import json

### constants imports

In [4]:
with open("../constants/columns.json", "r") as file: columns, years = json.load(file).values()

columns = namedtuple('Columns', ' '.join(columns.keys()))(**columns)
years = namedtuple('Years', re.sub(r'[\[\],\']','', str([*map(lambda y: f'y{y}', years)])))(*years)

### tools

In [5]:
YYYY = lambda txt, year : txt.replace('[YYYY]', str(year))

# Main

In [6]:
y_years = [2022]
x_years_increment = 2

filter_columns = [

   *[YYYY(columns.TD_div_TA_YYYY, year) for year in y_years],
   *[YYYY(columns.WC_div_TA_YYYY, year) for year in y_years],
   *[YYYY(columns.EBITDA_div_TA_YYYY, year) for year in y_years],
   *[YYYY(columns.lnTA_YYYY, year) for year in y_years],

   #*[YYYY(columns.z_zone_green_YYYY, year) for year in years],
   #*[YYYY(columns.z_zone_grey_YYYY, year) for year in years],
   #*[YYYY(columns.z_zone_red_YYYY, year) for year in years],
   *[YYYY(columns.z_cat_YYYY, year+x_years_increment) for year in y_years],
   
   columns.isFamiliar
]

In [7]:
data = pd.read_excel(IN_FILENAME)

In [8]:
df = data[filter_columns]

In [9]:
df.head()

,TD/TA 2022,WC/TA 2022,EBITDA/TA 2022,ln(TA) 2022,Z Cat 2024,isFamiliar
0,0.625639,0.191468,0.030919,19.710178,1,0
1,0.338930,0.373128,0.096122,18.584048,0,-1
2,0.140025,0.292908,0.178364,18.660925,0,0
3,0.221861,0.370248,0.090879,18.385202,0,1
4,0.635556,0.292989,0.064907,17.903484,1,1


# Save

In [10]:
match fmt:
    case "xlsx": df.to_excel(OUT_FILENAME, index=False)
    case "csv": df.to_csv(OUT_FILENAME, index=False)
    case _: raise Exception("Stop touching things!!")